# Python Package Choice

In [1]:
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 1.9 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import re
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline 
from textblob import TextBlob
import sklearn


## CHANGE PATH FOR YOU MACHINE

In [2]:
# Define the paths to the files
gift_card_path = 'Sentiment-analysis-with-LLM/raw_data/amazon_reviews_us_Gift_Card_v1_00.tsv'
major_appliances_path = 'Sentiment-analysis-with-LLM/raw_data/amazon_reviews_us_Major_Appliances_v1_00.tsv'

# Read the files into dataframes
df_gift_card = pd.read_csv(gift_card_path, sep='\t', error_bad_lines=False)
df_major_appliances = pd.read_csv(major_appliances_path, sep='\t', error_bad_lines=False)

# Display the first few rows of each dataframe to verify
# print("Gift Card Reviews DataFrame:")
# display(df_gift_card.head())
# print("\nMajor Appliances Reviews DataFrame:")
# display(df_major_appliances.head())

/var/folders/3h/r5l1t4656pbc6bzdv94m9wrm0000gn/T/ipykernel_31407/2187263467.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_gift_card = pd.read_csv(gift_card_path, sep='\t', error_bad_lines=False)
/var/folders/3h/r5l1t4656pbc6bzdv94m9wrm0000gn/T/ipykernel_31407/2187263467.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_major_appliances = pd.read_csv(major_appliances_path, sep='\t', error_bad_lines=False)
Skipping line 5583: expected 15 fields, saw 22
Skipping line 22814: expected 15 fields, saw 22
Skipping line 22883: expected 15 fields, saw 22
Skipping line 29872: expected 15 fields, saw 22
Skipping line 37242: expected 15 fields, saw 22
Skipping line 59693: expected 15 fields, saw 22



## Join both Datasets

In [3]:
# Concatenate the two dataframes vertically
df_combined = pd.concat([df_gift_card, df_major_appliances], ignore_index=True)

# Check the combined dataframe
print(df_combined.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245144 entries, 0 to 245143
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        245144 non-null  object
 1   customer_id        245144 non-null  int64 
 2   review_id          245144 non-null  object
 3   product_id         245144 non-null  object
 4   product_parent     245144 non-null  int64 
 5   product_title      245144 non-null  object
 6   product_category   245144 non-null  object
 7   star_rating        245144 non-null  int64 
 8   helpful_votes      245144 non-null  int64 
 9   total_votes        245144 non-null  int64 
 10  vine               245144 non-null  object
 11  verified_purchase  245144 non-null  object
 12  review_headline    245140 non-null  object
 13  review_body        245135 non-null  object
 14  review_date        245142 non-null  object
dtypes: int64(5), object(10)
memory usage: 28.1+ MB
None


In [ ]:
# df_combined['review_date'] = pd.to_datetime(df_combined['review_date'])

### TextBlob Test

In [4]:
text = "This product has been excellent in performance but I had issues with its customer service."
blob = TextBlob(text)
print(f"TextBlob Sentiment: {blob.sentiment}")

TextBlob Sentiment: Sentiment(polarity=1.0, subjectivity=1.0)


In [5]:
reviews = [
     "This product has been excellent in performance but I had issues with its customer service.",
    "Absolutely love this product, it works wonders!",
    "Terrible, did not work as advertised.",
    "Okay, but I've seen better performance from cheaper products.",
    "Amazing quality, exceeded my expectations.",
    "The worst customer service experience ever.",
    "Pretty good, but not worth the price.",
    "This product saved me so much time!",
    "Not happy with the purchase. It broke after a week.",
    "Excellent durability and performance. Highly recommend.",
    "Mediocre at best, I wouldn't buy it again."
]

for review in reviews:
    blob = TextBlob(review)
    print(f"Review: {review}\nTextBlob Sentiment: {blob.sentiment}\n")

Review: This product has been excellent in performance but I had issues with its customer service.
TextBlob Sentiment: Sentiment(polarity=1.0, subjectivity=1.0)

Review: Absolutely love this product, it works wonders!
TextBlob Sentiment: Sentiment(polarity=0.625, subjectivity=0.6)

Review: Terrible, did not work as advertised.
TextBlob Sentiment: Sentiment(polarity=-1.0, subjectivity=1.0)

Review: Okay, but I've seen better performance from cheaper products.
TextBlob Sentiment: Sentiment(polarity=0.5, subjectivity=0.5)

Review: Amazing quality, exceeded my expectations.
TextBlob Sentiment: Sentiment(polarity=0.6000000000000001, subjectivity=0.9)

Review: The worst customer service experience ever.
TextBlob Sentiment: Sentiment(polarity=-1.0, subjectivity=1.0)

Review: Pretty good, but not worth the price.
TextBlob Sentiment: Sentiment(polarity=0.26666666666666666, subjectivity=0.5666666666666668)

Review: This product saved me so much time!
TextBlob Sentiment: Sentiment(polarity=0.25, 

#### Apply To Our Data

In [6]:
# Define a function to compute the sentiment
def compute_sentiment(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None

# Apply the function to the 'review_body' column
df_combined['sentiment'] = df_combined['review_body'].apply(compute_sentiment)

# Optionally, you can print out some of the reviews and their calculated sentiment scores to check
for index, row in df_combined.head(10).iterrows():
    print(f"Review: {row['review_body']}\nTextBlob Sentiment: {row['sentiment']}\n")


Review: Great birthday gift for a young adult.
TextBlob Sentiment: Sentiment(polarity=0.3333333333333333, subjectivity=0.48333333333333334)

Review: It's an Amazon gift card and with over 982398324908 items I am sure you will find something you love.
TextBlob Sentiment: Sentiment(polarity=0.5, subjectivity=0.7444444444444445)

Review: Good
TextBlob Sentiment: Sentiment(polarity=0.7, subjectivity=0.6000000000000001)

Review: Fair
TextBlob Sentiment: Sentiment(polarity=0.7, subjectivity=0.9)

Review: I can't believe how quickly Amazon can get these into my hands!!  Thank you!
TextBlob Sentiment: Sentiment(polarity=0.6510416666666665, subjectivity=0.5)

Review: Perfect!  Nice & easy plus everyone loves them.  This is one gift you'll never have to return!
TextBlob Sentiment: Sentiment(polarity=0.7138888888888889, subjectivity=0.9444444444444445)

Review: excelent
TextBlob Sentiment: Sentiment(polarity=0.0, subjectivity=0.0)

Review: Great and Safe Gift Giving
TextBlob Sentiment: Sentiment(

### Hugging Face Transformers Test

In [7]:
nlp = pipeline("sentiment-analysis")
text = "This product has been excellent in performance but I had issues with its customer service."
result = nlp(text)
print(f"Hugging Face Sentiment: {result}")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9973146319389343}]


In [8]:
nlp = pipeline("sentiment-analysis")

reviews = [
    "This product has been excellent in performance but I had issues with its customer service.",
    "Absolutely love this product, it works wonders!",
    "Terrible, did not work as advertised.",
    "Okay, but I've seen better performance from cheaper products.",
    "Amazing quality, exceeded my expectations.",
    "The worst customer service experience ever.",
    "Pretty good, but not worth the price.",
    "This product saved me so much time!",
    "Not happy with the purchase. It broke after a week.",
    "Excellent durability and performance. Highly recommend.",
    "Mediocre at best, I wouldn't buy it again."
]

for review in reviews:
    result = nlp(review)
    print(f"Review: {review}\nHugging Face Sentiment: {result}\n")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Review: This product has been excellent in performance but I had issues with its customer service.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9973146319389343}]

Review: Absolutely love this product, it works wonders!
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998862743377686}]

Review: Terrible, did not work as advertised.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9994440674781799}]

Review: Okay, but I've seen better performance from cheaper products.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9877060055732727}]

Review: Amazing quality, exceeded my expectations.
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998855590820312}]

Review: The worst customer service experience ever.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.999789297580719}]

Review: Pretty good, but not worth the price.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9121016263961792}]

Review: This product saved me so mu

#### Apply To Our Data

In [12]:
# Initialize the sentiment analysis pipeline
nlp = pipeline("sentiment-analysis")

# Define a function to apply sentiment analysis
def apply_sentiment_analysis(review):
    try:
        result = nlp(review)
        return result
    except Exception as e:
        print(f"Error processing review: {e}")
        return None

# Apply the function to a subset of the 'review_body' column for demonstration
# Note: Applying this to a large dataset might be time-consuming
df_combined['huggingface_sentiment'] = df_combined['review_body'].head(100).apply(apply_sentiment_analysis)

# Optionally, print some results to verify
for index, row in df_combined.head(10).iterrows():
    print(f"Review: {row['review_body']}\nHugging Face Sentiment: {row['huggingface_sentiment']}\n")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Review: Great birthday gift for a young adult.
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.999884843826294}]

Review: It's an Amazon gift card and with over 982398324908 items I am sure you will find something you love.
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9996628761291504}]

Review: Good
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998161196708679}]

Review: Fair
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9997696280479431}]

Review: I can't believe how quickly Amazon can get these into my hands!!  Thank you!
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.998824417591095}]

Review: Perfect!  Nice & easy plus everyone loves them.  This is one gift you'll never have to return!
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998905658721924}]

Review: excelent
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998672008514404}]

Review: Great and Safe Gift Giving
Hugging Face Sentiment: [{'label'